<a href="https://colab.research.google.com/github/hscrown/madatpublicdata/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 임포트

In [1]:
!pip install lxml
!pip install datasets
!pip install haversine

import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from lxml import etree
from haversine import haversine
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# 함수 정의
반드시 순서대로 호출할 것

오늘 날짜 불러와서 문자열로 나타내는 함수

In [ ]:
def whats_the_date():
  what_date = datetime.now().strftime("%Y%m%d")
  what_time = datetime.now().strftime("%H%M")
  return what_date, what_time

기상청 좌표 데이터 전처리함수

In [ ]:
# 데이터셋 라이브러리를 사용하여 기상청 데이터셋 로드 - 로컬에서 안불러와도됨
def bring_weather_coor():
  dataset = load_dataset("hscrown/weather_api_info")
  kor_loc = pd.DataFrame(dataset['train'])
  # kor_loc = pd.read_csv('dataset.csv')
  kor_loc = kor_loc.iloc[:,:15] # 필요한 컬럼만 추출
  kor_loc = kor_loc.dropna() # 2단계와 3단계가 모두 존재하는 행만 추출

  return kor_loc

내 좌표를 넣으면 가장 가까운 기상청 x,y좌표로 변환하는 함수

In [ ]:
# 내 좌표를 기상청 x,y좌표로 변환

def find_nearest_grid(my_loc,kor_loc):
  grid = None
  min_distance = float('inf')  # 무한대 값으로 초기화

  for index, row in kor_loc.iterrows():
      # 각 격자 지점에 대한 튜플 (위도, 경도)을 생성
      grid_point = (row['위도(초/100)'], row['경도(초/100)'])

      # haversine 공식을 사용하여 거리계산
      distance = haversine(my_loc, grid_point)

      # 가장 가까운 거리를 찾으면 정보를 업데이트
      if distance < min_distance:
          min_distance = distance
          grid = row
          nx = grid['격자 X']
          ny = grid['격자 Y']

      return grid,nx,ny

내 위치와 가장가까운 공원, 박물관 찾아주는 함수정의

In [ ]:
def find_nearest_place(my_loc, df):
    min_distance = float('inf')  # 무한대 값으로 초기화
    lat, long = None, None  # 초기 위치 변수 선언

    for index, row in df.iterrows():
        # 각 격자 지점에 대한 튜플 (위도, 경도)을 생성
        grid_point = (row['LATITUDE'], row['LONGITUDE'])

        # haversine 공식을 사용하여 거리 계산
        distance = haversine(my_loc, grid_point)

        # 가장 가까운 거리를 찾으면 정보를 업데이트
        if distance < min_distance:
            min_distance = distance
            lat = row['LATITUDE']
            long = row['LONGITUDE']
            name = row['NAME']
            adres = row['ADRES']

    return name, lat, long, adres

좌표와 시간을 넣으면 기상정보를 제공하는 *함수*

In [ ]:
def get_weather_info(base_date,base_time,nx,ny):

  # 초단기실황데이터
  url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
  params ={
      'serviceKey': 'sX3JWddMWHJxC43fx9mqgcqSsbmAlTpoFTUPbnrE1Db5uVnEAs7gJIL4Z3tzW1u2S6UC+8/go3xYCnG2wDctAQ==',
      'pageNo': '1',
      'numOfRows': '1000',
      'dataType': 'XML',
      'base_date': base_date,
      'base_time': base_time,
      'nx': nx,
      'ny': ny
  }

  response = requests.get(url, params=params)
  root = etree.fromstring(response.content)

  # 데이터 파싱 및 추출
  # category = root.xpath('//category/text()')[0] # 0:강수형태, 2:습도, 3:기온, 4:풍속
  rain = root.xpath('//obsrValue/text()')[0] # 강수
  temp = root.xpath('//obsrValue/text()')[3] # 기온

  mapping = {
    '0': "비가 오고 있지 않습니다.",
    '1': "비 소식이 있습니다.",
    '2': "비 또는 눈이 내립니다.",
    '3': "눈이 오고 있습니다.",
    '4': "소나기가 옵니다.",
    '5': "빗방울이 떨어집니다.",
    '6': "빗방울과 눈날림이 있습니다.",
    '7': "눈날림이 있습니다."
  }

  rain = mapping.get(rain)

    # 초단기예보데이터
  url2 = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'


  response2 = requests.get(url2, params=params)
  root2 = etree.fromstring(response2.content)

  # 엘리먼트 선택
  items = root2.xpath('//item')

  # 딕셔너리로 만들기
  data = [{
      "baseDate": item.findtext("baseDate"),
      "baseTime": item.findtext("baseTime"),
      "category": item.findtext("category"),
      "fcstDate": item.findtext("fcstDate"),
      "fcstTime": item.findtext("fcstTime"),
      "fcstValue": item.findtext("fcstValue"),
      "nx": item.findtext("nx"),
      "ny": item.findtext("ny")
  } for item in items]

  # 데이터프레임으로 만들기
  df = pd.DataFrame(data)
  df = df[df['fcstDate'] == df['baseDate']] # 오늘 예측 값만
  # df = df[df['fcstTime'] == df['baseTime']]
  df

  sky_dict = {
      '1': "맑음",
      '2': "구름조금",
      '3': "구름많음",
      '4': "흐림"
  }

  # 30분뒤 하늘상태는
  df = df[df['category'] == 'SKY']['fcstValue'].map(sky_dict)
  sky = df.values[0]


  return rain,temp,sky

# print(f"Result Code: {resultCode}, Message: {resultMsg}")
# print(f"Number of Rows: {numOfRows}, Page Number: {pageNo}, Total Count: {totalCount}")
# print(f"Data Type: {dataType}, Base Date: {base_date}, Base Time: {base_time}")
# print(f"Coordinates: ({nx}, {ny}), Category: {category}, Observed Value: {obsrValue}")

공원데이터 전처리 코드

In [ ]:
def get_park_data():
  # API 요청
  start_point = 1
  end_point = 1000 # 최대 1000개까지만 호출 할 수 있음
  seoul_key = '57524f76506d656e3732636a52457a'


  url = f'http://openAPI.seoul.go.kr:8088/{seoul_key}/json/SearchParkInfoService/{start_point}/{end_point}/'

  park = requests.get(url).json()
  park.keys() # ['SearchParkInfoService']

  park = pd.DataFrame(park['SearchParkInfoService']['row'])
  # 컬럼명 변경
  park.rename(columns={'P_PARK':"NAME",'P_ADDR':"ADRES",'XCNTS':'LATITUDE','YDNTS':"LONGITUDE"},inplace=True)
  # ['LATITUDE'] 컬럼을 실수로 변경
  # 결측행 삭제
  park['LATITUDE'].replace('', np.nan, inplace=True)
  park['LONGITUDE'].replace('', np.nan, inplace=True)
  park = park.dropna()

  # ['LATITUDE'] 컬럼을 실수로 변경
  park['LATITUDE'] = park['LATITUDE'].astype(float)

  # ['LONGITUDE'] 컬럼을 실수로 변경
  park['LONGITUDE'] = park['LONGITUDE'].astype(float)

  return park

도서관 데이터 전처리 코드

In [ ]:
def get_lib_data():
  start_point = 1
  end_point = 1000 # 최대 1000개까지만 호출 할 수 있음
  api_key = '57524f76506d656e3732636a52457a'

  url = f'http://openAPI.seoul.go.kr:8088/{api_key}/json/SeoulLibraryTimeInfo/{start_point}/{end_point}/'
  url2 = f'http://openAPI.seoul.go.kr:8088/{api_key}/json/SeoulLibraryTimeInfo/1001/2000/'

  data = requests.get(url).json()
  data2 = requests.get(url2).json()

  data = pd.DataFrame(data['SeoulLibraryTimeInfo']['row'])
  data2 = pd.DataFrame(data2['SeoulLibraryTimeInfo']['row'])

  lib = pd.concat([data, data2])
  # 컬럼명 변경
  lib.rename(columns={'LBRRY_NAME':"NAME",'ADRES':"ADRES",'XCNTS':'LATITUDE','YDNTS':"LONGITUDE"},inplace=True)

  # ['LATITUDE'] 컬럼을 실수로 변경
  lib['LATITUDE'] = lib['LATITUDE'].astype(float)

  # ['LONGITUDE'] 컬럼을 실수로 변경
  lib['LONGITUDE'] = lib['LONGITUDE'].astype(float)

  return lib


박물관 데이터 전처리코드

In [ ]:

"""박물관 데이터 전처리 코드"""

def get_muse_data():
  muse = load_dataset("hscrown/seoul_museums")
  muse = pd.DataFrame(muse['train'])

  # 컬럼명 변경
  muse.rename(columns={'시설명':"NAME",'주소':"ADRES",'위도':'LATITUDE','경도':'LONGITUDE'},inplace=True)

  # ['LATITUDE'] 컬럼을 실수로 변경
  # 결측행 삭제
  muse['LATITUDE'].replace('', np.nan, inplace=True)
  muse['LONGITUDE'].replace('', np.nan, inplace=True)
  muse = muse.dropna()

  # ['LATITUDE'] 컬럼을 실수로 변경
  muse['LATITUDE'] = muse['LATITUDE'].astype(float)

  # ['LONGITUDE'] 컬럼을 실수로 변경
  muse['LONGITUDE'] = muse['LONGITUDE'].astype(float)

  return muse

# 실행 코드

In [ ]:
# 변수에 오늘날짜 담기
base_date, base_time = whats_the_date()
base_date, base_time

('20240504', '1819')

In [ ]:
# 변수에 내 위치 담기
my_loc = (37.566, 126.9784)

In [ ]:
# 기상청 x,y좌표 데이터 불러오기
kor_loc = bring_weather_coor()

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# 내 위치와 가장 가까운 위치 찾기
grid,nx,ny = find_nearest_grid(my_loc,kor_loc)

In [ ]:
# 내 위치는
grid[['1단계','2단계','3단계']]

1단계    서울특별시
2단계      종로구
3단계    청운효자동
Name: 2, dtype: object

In [ ]:
# 기상청 좌표
nx,ny

(60, 127)

In [ ]:
# 날짜와 시간, 좌표를 넣어서 날씨정보 획득
rain, temp, sky= get_weather_info(base_date,base_time,nx,ny)
rain, temp, sky

('비가 오고 있지 않습니다.', '24.1', '맑음')

In [ ]:
# 박물관정보 불러오기
muse = get_muse_data()

# 공원정보 불러오기
park = get_park_data()

# 도서관 정보 불러오기
lib = get_lib_data()

# 날씨 정보 불러오기
weather_info = {"rain":rain, "sky":sky, "temp":temp}

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# 날씨정보를 넣으면 날씨를 알려주고 장소 추천하는 함수
# 강수가 맑음이 아니거나 기온이 30도 이상이면 도서관이나 미술관에 가세요
def where_to_go(rain,temp,sky):
  if (rain != '비가 오고 있지 않습니다.' or float(temp) >= 30) :
    libname, liblat, liblong, libadres = find_nearest_place(my_loc,lib)
    musename, muselat, muselong, museadres = find_nearest_place(my_loc,muse)
    return weather_info,{"libname":libname,"liblat":liblat,"liblong":liblong,"libadres":libadres,"musename":musename,"muselat":muselat,"muselong":muselong,"museadres":museadres}
  else:
    parkname, parklat, parklong, parkadres = find_nearest_place(my_loc,park)
    return weather_info,{"parkname":parkname,"parklat":parklat,"parklong":parklong,"parkadres":parkadres}

In [ ]:
# 장소추천하기
where_to_go(rain,temp,sky)

({'rain': '비가 오고 있지 않습니다.', 'sky': '맑음', 'temp': '24.1'},
 {'parkname': '세종로공원',
  'parklat': 37.5735,
  'parklong': 126.9758875,
  'parkadres': '서울특별시 종로구 세종대로 189 (우) 03172'})

In [9]:
# -*- coding: utf-8 -*-
"""weather.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/hscrown/madatpublicdata/blob/main/weather.ipynb

# 패키지 임포트
"""

# !pip install lxml
# !pip install datasets
# !pip install haversine

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from lxml import etree
from haversine import haversine
from datasets import load_dataset

# 오늘 날짜와 시간 불러오기
what_date = datetime.now().strftime("%Y%m%d")
what_time = datetime.now().strftime("%H%M")

# 기상청 좌표 데이터 로드
dataset = load_dataset("hscrown/weather_api_info")
kor_loc = pd.DataFrame(dataset['train'])
kor_loc = kor_loc.iloc[:,:15]
kor_loc = kor_loc.dropna()

# 내 좌표 설정
my_loc = (37.566, 126.9784)

# 가장 가까운 기상청 x, y 좌표 찾기
grid, min_distance, nx, ny = None, float('inf'), None, None
for index, row in kor_loc.iterrows():
    grid_point = (row['위도(초/100)'], row['경도(초/100)'])
    distance = haversine(my_loc, grid_point)
    if distance < min_distance:
        min_distance = distance
        grid = row
        nx, ny = grid['격자 X'], grid['격자 Y']

# 기상 정보 가져오기
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params = {
    'serviceKey': 'sX3JWddMWHJxC43fx9mqgcqSsbmAlTpoFTUPbnrE1Db5uVnEAs7gJIL4Z3tzW1u2S6UC+8/go3xYCnG2wDctAQ==',
    'pageNo': '1',
    'numOfRows': '1000',
    'dataType': 'XML',
    'base_date': what_date,
    'base_time': what_time,
    'nx': nx,
    'ny': ny
}
response = requests.get(url, params=params)
root = etree.fromstring(response.content)
rain = root.xpath('//obsrValue/text()')[0]
temp = root.xpath('//obsrValue/text()')[3]
rain_mapping = {
    '0': "비가 오고 있지 않습니다.",
    '1': "비 소식이 있습니다.",
    '2': "비 또는 눈이 내립니다.",
    '3': "눈이 오고 있습니다.",
    '4': "소나기가 옵니다.",
    '5': "빗방울이 떨어집니다.",
    '6': "빗방울과 눈날림이 있습니다.",
    '7': "눈날림이 있습니다."
}
rain = rain_mapping.get(rain, "기상 정보 없음")

  # 초단기예보데이터
url2 = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'


response2 = requests.get(url2, params=params)
root2 = etree.fromstring(response2.content)

# 엘리먼트 선택
items = root2.xpath('//item')

# 딕셔너리로 만들기
data = [{
    "baseDate": item.findtext("baseDate"),
    "baseTime": item.findtext("baseTime"),
    "category": item.findtext("category"),
    "fcstDate": item.findtext("fcstDate"),
    "fcstTime": item.findtext("fcstTime"),
    "fcstValue": item.findtext("fcstValue"),
    "nx": item.findtext("nx"),
    "ny": item.findtext("ny")
} for item in items]

# 데이터프레임으로 만들기
df = pd.DataFrame(data)
df = df[df['fcstDate'] == df['baseDate']] # 오늘 예측 값만
# df = df[df['fcstTime'] == df['baseTime']]
df

sky_dict = {
    '1': "맑음",
    '2': "구름조금",
    '3': "구름많음",
    '4': "흐림"
}

# 30분뒤 하늘상태는
df = df[df['category'] == 'SKY']['fcstValue'].map(sky_dict)
sky = df.values[0]

# 날씨 정보에 따른 장소 추천 로직
if rain != '비가 오고 있지 않습니다.' or float(temp) >= 30:
    # 박물관 정보 불러오기
    muse = load_dataset("hscrown/seoul_museums")
    muse = pd.DataFrame(muse['train'])
    muse.rename(columns={'시설명':"NAME",'주소':"ADRES",'위도':'LATITUDE','경도':'LONGITUDE'}, inplace=True)
    muse['LATITUDE'].replace('', np.nan, inplace=True)
    muse['LONGITUDE'].replace('', np.nan, inplace=True)
    muse = muse.dropna()
    muse['LATITUDE'] = muse['LATITUDE'].astype(float)
    muse['LONGITUDE'] = muse['LONGITUDE'].astype(float)

    # 도서관 정보 불러오기
    api_key = '57524f76506d656e3732636a52457a'
    start_point = 1
    end_point = 1000
    url = f'http://openAPI.seoul.go.kr:8088/{api_key}/json/SeoulLibraryTimeInfo/{start_point}/{end_point}/'
    url2 = f'http://openAPI.seoul.go.kr:8088/{api_key}/json/SeoulLibraryTimeInfo/1001/2000/'

    data = requests.get(url).json()
    data2 = requests.get(url2).json()

    data = pd.DataFrame(data['SeoulLibraryTimeInfo']['row'])
    data2 = pd.DataFrame(data2['SeoulLibraryTimeInfo']['row'])

    lib = pd.concat([data, data2])
    lib.rename(columns={'LBRRY_NAME':"NAME",'ADRES':"ADRES",'XCNTS':'LATITUDE','YDNTS':"LONGITUDE"}, inplace=True)
    lib['LATITUDE'] = lib['LATITUDE'].astype(float)
    lib['LONGITUDE'] = lib['LONGITUDE'].astype(float)

    # 결합
    places = pd.concat([muse, lib])

    # 가장 가까운 위치 찾기
    min_distance, place_name, place_lat, place_long, place_adres = float('inf'), None, None, None, None
    for index, row in places.iterrows():
        point = (row['LATITUDE'], row['LONGITUDE'])
        distance = haversine(my_loc, point)
        if distance < min_distance:
            min_distance = distance
            place_name, place_lat, place_long, place_adres = row['NAME'], row['LATITUDE'], row['LONGITUDE'], row['ADRES']
    result = {"sky":sky, "rain": rain, "temp": temp, "place": "cultural","name": place_name, "latitude": place_lat, "longitude": place_long, "address": place_adres}
else:
    park_url = 'http://openAPI.seoul.go.kr:8088/57524f76506d656e3732636a52457a/json/SearchParkInfoService/1/1000/'
    park_data = requests.get(park_url).json()['SearchParkInfoService']['row']
    park = pd.DataFrame(park_data)
    park.rename(columns={'P_PARK':"NAME",'P_ADDR':"ADRES",'XCNTS':'LATITUDE','YDNTS':"LONGITUDE"}, inplace=True)
    park['LATITUDE'].replace('', np.nan, inplace=True)
    park['LONGITUDE'].replace('', np.nan, inplace=True)
    park = park.dropna()
    park['LATITUDE'] = park['LATITUDE'].astype(float)
    park['LONGITUDE'] = park['LONGITUDE'].astype(float)
    min_distance, park_name, park_lat, park_long, park_adres = float('inf'), None, None, None, None
    for index, row in park.iterrows():
        point = (row['LATITUDE'], row['LONGITUDE'])
        distance = haversine(my_loc, point)
        if distance < min_distance:
            min_distance = distance
            park_name, park_lat, park_long, park_adres = row['NAME'], row['LATITUDE'], row['LONGITUDE'], row['ADRES']
    result = {"sky":sky, "rain": rain, "temp": temp, "place": "park","name": park_name, "latitude": park_lat, "longitude": park_long, "address": park_adres}

# 결과 출력
print(result)


Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


{'sky': '흐림', 'rain': '비 소식이 있습니다.', 'temp': '18.4', 'place': 'cultural', 'name': '서울도서관', 'latitude': 37.5663174209601, 'longitude': 126.977829174031, 'address': '서울특별시 중구 세종대로 110 태평로1가'}
